In [1]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
import sys
sys.path.append("/Users/ibragimzhussup/Desktop/APA_Lab/src/modules")
from preprocess import PreprocessAPA
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv("/Users/ibragimzhussup/Desktop/APA_Lab/prep_t.csv")
df = df.dropna()

In [4]:
prep = PreprocessAPA()
train, test = prep.split_data(df, test_val=False, n_samples_train=500)

In [16]:
# Assuming 'train' and 'test' are pandas DataFrames with columns 'text' for the features and 'label' for the labels
X_train = train['text']
y_train = train['labels']
X_test = test['text']
y_test = test['labels']

# Step 1: Mapping the original labels for broad categorization
y_train_step1 = y_train.replace({'interview': 'others', 'comment': 'others'})
y_test_step1 = y_test.replace({'interview': 'others', 'comment': 'others'})

In [17]:
# Pipeline for step 1: Broad categorization (letter, non_relevant, others)
model_step1 = make_pipeline(TfidfVectorizer(use_idf=True, max_features=5000, norm='l2', sublinear_tf=True), 
                            SVC(kernel='linear', probability=True))
model_step1.fit(X_train, y_train_step1)
y_pred_step1 = model_step1.predict(X_test)

In [18]:
print(classification_report(y_test_step1, y_pred_step1))

              precision    recall  f1-score   support

 NONRELEVANT       1.00      0.78      0.88    168485
      letter       0.98      0.92      0.95      4050
      others       0.04      0.98      0.08      1697

    accuracy                           0.79    174232
   macro avg       0.67      0.89      0.64    174232
weighted avg       0.99      0.79      0.87    174232



In [21]:
# Step 2: Train a classifier for the 'others' category
train_others = train[y_train.isin(['interview', 'comment'])]
X_train_others = train_others['text']
y_train_others = train_others['labels']

test_others = test[y_pred_step1 == 'others']
X_test_others = test_others['text']
y_test_others = test_others['labels']

In [23]:
y_train_others.unique()

array(['comment', 'interview'], dtype=object)

In [25]:
# Pipeline for step 2: Classification within 'others'
model_step2 = make_pipeline(
    TfidfVectorizer(use_idf=True, max_features=5000, norm='l2', sublinear_tf=True),
    SVC(kernel='linear')
)
model_step2.fit(X_train_others, y_train_others)
y_pred_step2 = model_step2.predict(X_test_others)

In [26]:
# Combine predictions
final_predictions = y_pred_step1.copy()
final_predictions[test.index.isin(test_others.index)] = y_pred_step2

In [29]:
print(classification_report(y_test, final_predictions))

              precision    recall  f1-score   support

 NONRELEVANT       1.00      0.78      0.88    168485
     comment       0.03      0.98      0.06       629
   interview       0.06      0.94      0.11      1068
      letter       0.98      0.92      0.95      4050

    accuracy                           0.79    174232
   macro avg       0.52      0.91      0.50    174232
weighted avg       0.99      0.79      0.87    174232

